In [ ]:
using Revise
using Echidna
using Plots
gr();

Core structs: 
```
struct Problem
    nobjs::Int64
    directions::Vector{Bool}
    nvars::Int64
    var_types::Vector{MOGA_Type},
    eval_fn::Function
end

mutable struct Solution
    problem::Problem
    x::Vector{Float64}
    evaluated::Bool
    objectives::Vector{Float64}
    crowding_distance::Float64
    rank::Int64
end
```

# Testing archive

In [ ]:
prob = Problem(2, [false, false], 5, [MOGA_Real(0,10) for i in 1:5], x->sum(x)) # true means MINIMIZE, so we're maximizing here
sols = Vector{Solution}()
N = 10
for i=0:N
    r = i*(pi/2)/N
    objs = [cos(r), sin(r)]
    push!(sols, Solution(prob, [0.0], true, objs, 0.0, 0))
end

In [ ]:
archive = Archive(compare_pareto_dominance, Vector{Solution}())
insert_solutions!(archive, sols)
x = [s.objectives[1] for s in archive.solutions]
y = [s.objectives[2] for s in archive.solutions]
scatter(x, y, size=(300,300), aspect_ratio=:equal, legend=false)

In [ ]:
s1 = Solution(prob, [0.0], true, [.5, .5], 0.0, 0)
s2 = Solution(prob, [0.0], true, [.9, .9], 0.0, 0)

In [ ]:
insert_solutions!(archive, [s1])
x = [s.objectives[1] for s in archive.solutions]
y = [s.objectives[2] for s in archive.solutions]
scatter(x, y, size=(300,300), aspect_ratio=:equal, legend=false)

In [ ]:
insert_solutions!(archive, [s2])
x = [s.objectives[1] for s in archive.solutions]
y = [s.objectives[2] for s in archive.solutions]
scatter(x, y, size=(300,300), aspect_ratio=:equal, legend=false)

# Testing non-dominated sort

In [ ]:
prob = Problem(2, [false, false], 5, [MOGA_Real(0,10) for i in 1:5], x->sum(x)) # true means MINIMIZE, so we're maximizing here
sols = Vector{Solution}()
N = 10
for i=0:N
    r = i*(pi/2)/N
    objs = [cos(r), sin(r)]
    push!(sols, Solution(prob, [0.0], true, objs, 0.0, 0))
    push!(sols, Solution(prob, [0.0], true, .9*objs, 0.0, 0))
    push!(sols, Solution(prob, [0.0], true, .8*objs, 0.0, 0))
end
Echidna.nondominated_sort(sols)

In [ ]:
x = [s.objectives[1] for s in sols]
y = [s.objectives[2] for s in sols]
r = [s.rank for s in sols]
cdist = [s.crowding_distance for s in sols]
cdist[isinf.(cdist)] = 0
p1 = scatter(x, y, zcolor=r, aspect_ratio=:equal, legend=false, title="rank")
p2 = scatter(x, y, zcolor=cdist, aspect_ratio=:equal, legend=false, title="crowding distance")
plot(p1, p2, layout=(1, 2))

# Testing mutation and crossover operators
The default NSGA II algorithm uses the `PM` mutation operator and `SBX` crossover operator, so those were the first two implemented

In [ ]:
prob = Problem(2, [false, false], 5, [MOGA_Real(0,10) for i in 1:5], x->sum(x)) # true means MINIMIZE, so we're maximizing here
p1 = Solution(prob, [3. for i in 1:5], true, [1.0, 0.0], 0.0, 0);
p2 = Solution(prob, [6. for i in 1:5], true, [1.0, 0.0], 0.0, 0);

In [ ]:
c = PM(p1).x

In [ ]:
mut_vals = [Echidna.pm_mutation(2.5, 0.0, 10.0, 10.0) for i in 1:100000];

In [ ]:
histogram(mut_vals)

In [ ]:
c1, c2 = SBX(p1, p2)
show(c1.x)
print("\n")
show(c2.x)

# Experiment with ZTD1

In [ ]:
nobjs = 2
nvars = 30
zdt1_problem = Problem(
    2,
    [false for i in 1:nobjs],
    nvars,
    [MOGA_Real(0.0, 1.0) for i in 1:30],
    ZDT1
)
pop_size = 100
pop = [random_candidate(zdt1_problem) for i in 1:pop_size]
for i in 1:pop_size
    evaluate!(pop[i])
end

archive = Archive(compare_pareto_dominance, Vector{Solution}())
insert_solutions!(archive, pop)

In [ ]:
xp = [c.objectives[1] for c in pop]
yp = [c.objectives[2] for c in pop]
xa = [c.objectives[1] for c in archive.solutions]
ya = [c.objectives[2] for c in archive.solutions]
scatter([xp,xa] , [yp, ya])

In [ ]:
Echidna.nondominated_sort(pop)

In [ ]:
x = [s.objectives[1] for s in pop]
y = [s.objectives[2] for s in pop]
r = [s.rank for s in pop]
cdist = [s.crowding_distance for s in pop]
cdist[isinf.(cdist)] = 0
p1 = scatter(x, y, zcolor=r, aspect_ratio=:equal, legend=false, title="rank")
p2 = scatter(x, y, zcolor=cdist, aspect_ratio=:equal, legend=false, title="crowding distance")
plot(p1, p2, layout=(1, 2), aspect_ratio=:none)

## Testing the tournament selector
What we want to see is points selected with a bias towards the pareto frontier. 

In [ ]:
parents = [Echidna.tournament_selector(pop, 2, dominance=Echidna.nondominated_cmp) for i in 1:50];
xn = [c.objectives[1] for c in parents]
yn = [c.objectives[2] for c in parents]
xp = [c.objectives[1] for c in pop]
yp = [c.objectives[2] for c in pop]
scatter([xp,xn] , [yp, yn])

## Testing sorting by nondominated_cmp
Echidna.nondominated_cmp returns -1, 0, or 1, but we need a binary result for sorting

In [ ]:
k(c1, c2) = Echidna.nondominated_cmp(c1, c2) < 0

In [ ]:
sorted_pop = sort(pop, lt=k);
plot(1:100, [p.rank for p in sorted_pop])


# Testing NSGA II
```
@with_kw mutable struct NSGAII <: Algorithm
    problem::Problem
    eval_fn::Function
    population_size::Int64
    n_iters::Int64
end
```

In [ ]:
nobjs = 2
nvars = 30
zdt1_problem = Problem(
    2,
    [false for i in 1:nobjs],
    nvars,
    [MOGA_Real(0.0, 1.0) for i in 1:nvars],
    ZDT1
)

algo = NSGAII(zdt1_problem, ZDT1, 100, 50)

In [ ]:
@time sols = Echidna.nsgaii(algo);
xinit = [c.objectives[1] for c in sols]
yinit = [c.objectives[2] for c in sols]
scatter(xinit, yinit)